# Sửa lỗi Duplicate IDs trong Child Chunks
Notebook này tạo lại child chunks với IDs unique hoàn toàn

In [1]:
# Import dependencies
import sys
import os
import json

# Đường dẫn tới thư mục gốc
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(project_root)

from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import numpy as np
from utils.load_chunks_json import load_chunks_from_json
from utils.save_chunks_json import save_chunks_to_json

print(f"Đã thêm đường dẫn: {project_root}")
print(f"Thư mục hiện tại: {os.getcwd()}")


Đã thêm đường dẫn: d:\Documents\TDTU\Kiến tập\RAG_GeneralSubject
Thư mục hiện tại: d:\Documents\TDTU\Kiến tập\RAG_GeneralSubject\data\LichSuDang


In [2]:
# Load parent chunks
dense_chunk = load_chunks_from_json("Lich_Su_Dang_Dense.json")
print(f"Đã load {len(dense_chunk)} parent chunks")

# Khởi tạo embedding model
embedding_model = SentenceTransformer("AITeamVN/Vietnamese_Embedding")
embedding_model.max_seq_length = 2048
print(f"Đã khởi tạo embedding model: {embedding_model.get_sentence_embedding_dimension()} dimensions")

Đã load 347 parent chunks
Đã khởi tạo embedding model: 1024 dimensions


In [3]:
def create_parent_document_chunks_unique(parent_chunks, embedding_model, child_chunk_size=450, overlap=75):
    """
    Tạo child chunks từ parent chunks với embedding và IDs hoàn toàn unique
    
    Args:
        parent_chunks: Danh sách parent chunks từ file Dense.json
        embedding_model: Model embedding để tạo vector cho child chunks
        child_chunk_size: Kích thước tối đa của mỗi child chunk (ký tự)
        overlap: Độ overlap giữa các child chunks
    
    Returns:
        child_chunks: Danh sách child chunks với embedding
    """
    child_chunks = []
    global_child_counter = 0  # Đảm bảo ID hoàn toàn unique
    
    # Khởi tạo text splitter cho child chunks
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=child_chunk_size,
        chunk_overlap=overlap,
        separators=["\n\n", "\n", ".", "!", "?", " ", ""]
    )
    
    for parent_chunk in parent_chunks:
        parent_id = parent_chunk['id']
        content = parent_chunk['metadata']['content']
        
        # Chỉ chunk những parent có content dài > child_chunk_size
        if len(content) <= child_chunk_size:
            # Nếu parent đủ ngắn, giữ nguyên nhưng sử dụng global counter
            child_chunk = {
                "id": f"LSD_child_{global_child_counter:05d}",  # Format: LSD_child_00001
                "values": embedding_model.encode(content).tolist(),
                "metadata": {
                    **parent_chunk['metadata'],  # Copy toàn bộ metadata từ parent
                    "parent_id": parent_id,
                    "child_index": 0,
                    "global_child_id": global_child_counter
                }
            }
            child_chunks.append(child_chunk)
            global_child_counter += 1
        else:
            # Chunk content thành nhiều phần nhỏ
            sub_contents = splitter.split_text(content)
            
            for i, sub_content in enumerate(sub_contents):
                child_chunk = {
                    "id": f"LSD_child_{global_child_counter:05d}",  # Format: LSD_child_00001
                    "values": embedding_model.encode(sub_content.strip()).tolist(),
                    "metadata": {
                        **parent_chunk['metadata'],  # Copy toàn bộ metadata từ parent
                        "content": sub_content.strip(),  # Override content với sub_content
                        "parent_id": parent_id,
                        "child_index": i,
                        "tokens": len(sub_content.strip()),  # Update token count
                        "global_child_id": global_child_counter
                    }
                }
                child_chunks.append(child_chunk)
                global_child_counter += 1
    
    return child_chunks

print("Đã định nghĩa hàm create_parent_document_chunks_unique")

Đã định nghĩa hàm create_parent_document_chunks_unique


In [4]:
# Tạo child chunks với IDs hoàn toàn unique
print("TẠOA CHILD CHUNKS VỚI IDs HOÀN TOÀN UNIQUE")
print("="*50)

child_chunks_unique = create_parent_document_chunks_unique(
    parent_chunks=dense_chunk,
    embedding_model=embedding_model,
    child_chunk_size=450,  # Sử dụng tham số tối ưu
    overlap=75
)

print(f"Hoàn thành! Đã tạo {len(child_chunks_unique)} child chunks")

# Kiểm tra IDs có unique không
chunk_ids_unique = [chunk['id'] for chunk in child_chunks_unique]
unique_ids_set = set(chunk_ids_unique)

print(f"\nKIỂM TRA UNIQUE:")
print(f"Tổng số chunks: {len(child_chunks_unique)}")
print(f"Số IDs unique: {len(unique_ids_set)}")

if len(chunk_ids_unique) == len(unique_ids_set):
    print("✓ Tất cả IDs đều hoàn toàn unique!")
else:
    print(f"⚠ Vẫn có {len(chunk_ids_unique) - len(unique_ids_set)} IDs bị duplicate")

# Hiển thị một số IDs mẫu
print(f"\nMẫu IDs mới:")
for i in range(min(10, len(child_chunks_unique))):
    chunk = child_chunks_unique[i]
    print(f"  {chunk['id']} (parent: {chunk['metadata']['parent_id']})")

TẠOA CHILD CHUNKS VỚI IDs HOÀN TOÀN UNIQUE


c:\Users\Duongw\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\xlm_roberta\modeling_xlm_roberta.py:371: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Hoàn thành! Đã tạo 1971 child chunks

KIỂM TRA UNIQUE:
Tổng số chunks: 1971
Số IDs unique: 1971
✓ Tất cả IDs đều hoàn toàn unique!

Mẫu IDs mới:
  LSD_child_00000 (parent: LSD_chuongnhapmon_0_0_0)
  LSD_child_00001 (parent: LSD_chuongnhapmon_0_0_0)
  LSD_child_00002 (parent: LSD_chuongnhapmon_0_0_0)
  LSD_child_00003 (parent: LSD_chuongnhapmon_0_0_0)
  LSD_child_00004 (parent: LSD_chuongnhapmon_0_0_0)
  LSD_child_00005 (parent: LSD_chuongnhapmon_0_0_0)
  LSD_child_00006 (parent: LSD_chuongnhapmon_0_0_1)
  LSD_child_00007 (parent: LSD_chuongnhapmon_I_0_0)
  LSD_child_00008 (parent: LSD_chuongnhapmon_0_0_0)
  LSD_child_00009 (parent: LSD_chuongnhapmon_0_0_0)


In [6]:
# Lưu file mới
output_file = "Lich_Su_Dang_Child_Dense_Unique.json"

# Lưu trực tiếp bằng json
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(child_chunks_unique, f, ensure_ascii=False, indent=2)

print(f"Đã lưu {len(child_chunks_unique)} child chunks vào file: {output_file}")
print("Sẵn sàng để upsert lại vào Pinecone!")

# Thống kê nhanh
child_content_lengths = [len(chunk['metadata']['content']) for chunk in child_chunks_unique]
print(f"\nTHỐNG KÊ:")
print(f"Độ dài trung bình: {np.mean(child_content_lengths):.2f} ký tự")
print(f"Độ dài min: {min(child_content_lengths)} ký tự")
print(f"Độ dài max: {max(child_content_lengths)} ký tự")

Đã lưu 1971 child chunks vào file: Lich_Su_Dang_Child_Dense_Unique.json
Sẵn sàng để upsert lại vào Pinecone!

THỐNG KÊ:
Độ dài trung bình: 327.17 ký tự
Độ dài min: 1 ký tự
Độ dài max: 450 ký tự
